In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

import rasterio

import warnings
warnings.filterwarnings("ignore")

In [27]:
# Define torch dataset Class
class Dataset(Dataset):
    def __init__(self,folder_path,dataset_file,sen2_amount=1,sen2_tile="all"):
        
        # define filepaths
        self.folder_path = folder_path
        # read file
        self.df = pd.read_pickle(dataset_file)
        # set amount of sen2 pictures that should be returned
        self.sen2_amount = sen2_amount
        
        # filter for sen2 tile
        if sen2_tile!="all":
            self.df = self.df[self.df["sen2_tile"]==sen2_tile]
            
        # clear up DF
        self.df = self.df[self.df["sen2_no"]>2]
        try:
            self.df = self.df.drop(labels=["level_0"], axis=1)
        except KeyError:
            pass
        self.df = self.df.reset_index()
        
    def __len__(self):
        """
        Returns length of data
        """
        return(len(self.df))
    
 
    def __getitem__(self,idx):
        
        current = self.df.iloc[idx]
        spot6_file = current["spot6_filenames"]
        sen2_files = current["sen2_filenames"]
        
        """READ SPOT6"""
        #with rasterio.open(self.folder_path+"y/"+spot6_file) as dataset:
        spot6 = rasterio.open(self.folder_path+"y/"+spot6_file).read()

    
        """READ SEN2 SERIES"""
        # read first file
        sen2 = rasterio.open(self.folder_path+"x/"+sen2_files[0]).read()
        
        if self.sen2_amount>1:
            # read following sen2 and stack
            count=1
            for sen2_file in sen2_files[1:]:
                # read file as array
                sen2_following = rasterio.open(self.folder_path+"x/"+sen2_file).read()
                # stack to previous images
                sen2 = np.concatenate([sen2, sen2_following])

                # break if all wanted files loaded
                count=count+1
                if count==self.sen2_amount:
                    break
            # if final count not yet reached, repeat last chip until enough are there
            while count<self.sen2_amount:
                sen2 = np.concatenate([sen2, sen2_following])
                count=count+1
        
        # transform to tensor
        sen2  = torch.from_numpy(sen2)
        spot6 = torch.from_numpy(spot6)
        sen2 = sen2.float()
        spot6 = spot6.float()
        
        #print(len(sen2_files),sen2.size())
        
        # define transformer
        transform_spot = transforms.Compose([transforms.Normalize(mean=[479.0, 537.0, 344.0], std=[430.0, 290.0, 229.0]) ])
        # dynamically define transform to reflect shape of tensor
        trans_mean,trans_std = [78.0, 91.0, 62.0]*self.sen2_amount,[36.0, 28.0, 30.0]*self.sen2_amount
        transform_sen = transforms.Compose([transforms.Normalize(mean=trans_mean, std= trans_std)])
        # perform transform
        sen2  = transform_sen(sen2)
        spot6 = transform_spot(spot6)
        
        # return result
        return(spot6,sen2)


In [59]:
dataset = Dataset("data_f4/","data_f4_pkls/df_saved_images.pkl",sen2_amount=4,sen2_tile="all")
loader  = DataLoader(dataset,batch_size=64, shuffle=True, num_workers=0,pin_memory=True,
                    drop_last=True,prefetch_factor=2)

In [63]:
dataset.df

,level_0,index,x,y,geometry,index_right,name,min,max,other_acq,geom,type,other_valid_acq,spot6_validity,sen2_no,sen2_filenames,sen2_tile,spot6_filenames
0,0,504,126000.0,6840000.0,POINT (126000.000 6840000.000),1979,ORT_2018_0126_6843_LA93_8Bits.jp2,1,144,"{33: [2018-04-03 00:00:00, 'SENTINEL2A_2018040...","POLYGON ((126000.000 6843000.000, 126000.000 6...",train,"{33: [2018-04-03 00:00:00, 'SENTINEL2A_2018040...",True,6,[SENTINEL2A_20180520-112543-431_L2A_T30UUU_C_V...,T30UUU,ORT_2018_0126_6843_LA93_8Bits_126000.0_6840000...
1,1,505,126000.0,6840750.0,POINT (126000.000 6840750.000),1979,ORT_2018_0126_6843_LA93_8Bits.jp2,1,144,"{33: [2018-04-03 00:00:00, 'SENTINEL2A_2018040...","POLYGON ((126000.000 6843000.000, 126000.000 6...",train,"{33: [2018-04-03 00:00:00, 'SENTINEL2A_2018040...",True,6,[SENTINEL2A_20180520-112543-431_L2A_T30UUU_C_V...,T30UUU,ORT_2018_0126_6843_LA93_8Bits_126000.0_6840750...
2,2,506,126000.0,6841500.0,POINT (126000.000 6841500.000),1979,ORT_2018_0126_6843_LA93_8Bits.jp2,1,144,"{33: [2018-04-03 00:00:00, 'SENTINEL2A_2018040...","POLYGON ((126000.000 6843000.000, 126000.000 6...",train,"{33: [2018-04-03 00:00:00, 'SENTINEL2A_2018040...",True,5,[SENTINEL2B_20180902-112716-061_L2A_T30UUU_C_V...,T30UUU,ORT_2018_0126_6843_LA93_8Bits_126000.0_6841500...
3,3,507,126000.0,6842250.0,POINT (126000.000 6842250.000),1979,ORT_2018_0126_6843_LA93_8Bits.jp2,1,144,"{33: [2018-04-03 00:00:00, 'SENTINEL2A_2018040...","POLYGON ((126000.000 6843000.000, 126000.000 6...",train,"{33: [2018-04-03 00:00:00, 'SENTINEL2A_2018040...",True,5,[SENTINEL2B_20180902-112716-061_L2A_T30UUU_C_V...,T30UUU,ORT_2018_0126_6843_LA93_8Bits_126000.0_6842250...
4,4,508,126000.0,6843000.0,POINT (126000.000 6843000.000),1979,ORT_2018_0126_6843_LA93_8Bits.jp2,1,144,"{33: [2018-04-03 00:00:00, 'SENTINEL2A_2018040...","POLYGON ((126000.000 6843000.000, 126000.000 6...",train,"{33: [2018-04-03 00:00:00, 'SENTINEL2A_2018040...",True,6,[SENTINEL2A_20180520-112543-431_L2A_T30UUU_C_V...,T30UUU,ORT_2018_0126_6843_LA93_8Bits_126000.0_6843000...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45375,68116,82642,401250.0,6828000.0,POINT (401250.000 6828000.000),3334,ORT_2018_0399_6831_LA93_8Bits.jp2,1,159,"{159: [2018-09-26 00:00:00, 'SENTINEL2B_201809...","POLYGON ((399000.000 6831000.000, 399000.000 6...",train,"{159: [2018-09-26 00:00:00, 'SENTINEL2B_201809...",True,9,[SENTINEL2B_20180926-110028-459_L2A_T30UXU_C_V...,T30UXU,ORT_2018_0399_6831_LA93_8Bits_401250.0_6828000...
45376,68117,82643,401250.0,6828750.0,POINT (401250.000 6828750.000),3334,ORT_2018_0399_6831_LA93_8Bits.jp2,1,159,"{159: [2018-09-26 00:00:00, 'SENTINEL2B_201809...","POLYGON ((399000.000 6831000.000, 399000.000 6...",train,"{159: [2018-09-26 00:00:00, 'SENTINEL2B_201809...",True,7,[SENTINEL2B_20180926-110028-459_L2A_T30UXU_C_V...,T30UXU,ORT_2018_0399_6831_LA93_8Bits_401250.0_6828750...
45377,68118,82644,401250.0,6829500.0,POINT (401250.000 6829500.000),3334,ORT_2018_0399_6831_LA93_8Bits.jp2,1,159,"{159: [2018-09-26 00:00:00, 'SENTINEL2B_201809...","POLYGON ((399000.000 6831000.000, 399000.000 6...",train,"{159: [2018-09-26 00:00:00, 'SENTINEL2B_201809...",True,7,[SENTINEL2B_20180926-110028-459_L2A_T30UXU_C_V...,T30UXU,ORT_2018_0399_6831_LA93_8Bits_401250.0_6829500...
45378,68119,82645,401250.0,6830250.0,POINT (401250.000 6830250.000),3334,ORT_2018_0399_6831_LA93_8Bits.jp2,1,159,"{159: [2018-09-26 00:00:00, 'SENTINEL2B_201809...","POLYGON ((399000.000 6831000.000, 399000.000 6...",train,"{159: [2018-09-26 00:00:00, 'SENTINEL2B_201809...",True,9,[SENTINEL2B_20180926-110028-459_L2A_T30UXU_C_V...,T30UXU,ORT_2018_0399_6831_LA93_8Bits_401250.0_6830250...


In [58]:
%%timeit
for i in loader:
    a,b = i
    break

2.4 s ± 96.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
